In [1]:
# start 27.10.20 20:41

import numpy as np
import random
import math


In [2]:
def genDat(n,l):
    rotL  = lambda x: x[1:] + [x[0]]
    
    rand  = lambda y: map(lambda z: random.choice([0,1]), [0] * y)
    
    x = map(rand, [l] * n)
    y = map(rotL, x)
    
    return zip(x,y)

In [3]:
def mat2(n,m,f):
    return [[f() for j in range(m)] for i in range(n)]

def mat1(n,f):
    return [f() for i in range(n)]

def dot_m_v(m1, v1):
    v = [0] * len(m1)
    for i,em1 in enumerate(m1):
        for j in range(len(em1)):
            v[i] += em1[j] * v1[j]
            
    return v

def dot_mT_v(m1, v1):
    v = [0] * len(m1[0])
    for i,em1 in enumerate(m1):
        for j in range(len(em1)):
            #v[j] += 
            v[j] += m1[i][j] * v1[i]
            
    return v

def sum_v(v1,v2):
    v = [0] * len(v1)
    for i in range(len(v1)):
        v[i] = v1[i] + v2[i]
        
    return v

def minus_v(v1,v2):
    v = [0] * len(v1)
    for i in range(len(v1)):
        v[i] = v1[i] - v2[i]
        
    return v

def mult_v_c(v,c):
    return [v[i] * c for i in range(len(v))]

def sqr_v(v):
    return [v[i] * v[i] for i in range(len(v))]

def mult_m2(m1,c):
    return [[m1[i][j] * c for j in range(len(m1[i]))] for i in range(len(m1))]

def mult_m3(m1,c):
    m = []
    for i in range(len(m1)):
        mx = [[0 for k in range(len(m1[i][0]))] for j in range(len(m1[i]))]
        for j in range(len(m1[i])):
            for k in range(len(m1[i][j])):                
                mx[j][k] = m1[i][j][k] * c
        m += [mx]
        
    return m



def minus_m2(m1,m2):
    m = [[m1[i][j] - m2[i][j] for j in range(len(m1[i]))] for i in range(len(m1))]
        
    return m

def minus_m3(m1,m2):
    m = []
    for i in range(len(m1)):
        mx = [[0 for k in range(len(m1[i][j]))] for j in range(len(m1[i]))]
        for j in range(len(m1[i])):
            for k in range(len(m1[i][j])):                
                mx[j][k] = m1[i][j][k] - m2[i][j][k]
        m += [mx]
        
    return m

def mult_v(v1,v2):
    v = [0] * len(v1)
    for i in range(len(v1)):
        v[i] = v1[i] * v2[i]
        
    return v

def outer_v_v(v1,v2):
    m = [[0 for j in range(len(v2))] for i in range(len(v1))]
    
    for i in range(len(v1)):
        for j in range(len(v2)):
            m[i][j] = v1[i] * v2[j]            
            
    return m
        
def outer_v_v_T(v1,v2):
    m = [[0 for j in range(len(v1))] for i in range(len(v2))]
    
    for i in range(len(v1)):
        for j in range(len(v2)):
            m[j][i] = v1[i] * v2[j]            
            
    return m   


In [4]:
-

In [9]:
dat = genDat(100000,3)

layers = [[3,3,3], [3,5,3], [3,4,5,3]]

for ll in layers:
    print "ll = {}".format(ll)
    nn = NN(ll)

    nn.test(dat[:5])
    print "TRAIN"
    nn.train(dat)
    nn.test(dat[:5])

ll = [3, 3, 3]
x = [0, 0, 0], y = [0, 0, 0], out = [0.7820289566985048, 0.8436844931990392, 0.8278542741838505], e = 1.41729161165
x = [1, 1, 1], y = [1, 1, 1], out = [0.8437402383275507, 0.8772978761221797, 0.8928779596956636], e = 0.225716760213
x = [1, 1, 1], y = [1, 1, 1], out = [0.8437402383275507, 0.8772978761221797, 0.8928779596956636], e = 0.225716760213
x = [0, 1, 0], y = [1, 0, 0], out = [0.8087323272005391, 0.8630808883835368, 0.8601416791414186], e = 1.23342435956
x = [1, 0, 0], y = [0, 0, 1], out = [0.8175583091297459, 0.8639080365581308, 0.8665706061177115], e = 1.19688850258
TRAIN
x = [0, 0, 0], y = [0, 0, 0], out = [0.05552586403483687, 0.04368714377684799, 0.02803939952584756], e = 0.0760124728842
x = [1, 1, 1], y = [1, 1, 1], out = [0.9527553239168092, 0.9644062519613316, 0.9751935936093755], e = 0.0641430597622
x = [1, 1, 1], y = [1, 1, 1], out = [0.9527553239168092, 0.9644062519613316, 0.9751935936093755], e = 0.0641430597622
x = [0, 1, 0], y = [1, 0, 0], out = [0.9